# Brain Tumor Detection

### For Google Colab Users

In [1]:
#@title ## Mount Your Google Drive & Map your Directory
#@markdown Please run this cell (`Ctrl+Enter` or `Shift+Enter`) and follow the steps printed bellow.
import os

from google.colab import drive
drive.mount('/content/gdrive')

# check Google Drive is mounted
if not os.path.isdir("/content/gdrive"):
  raise FileNotFoundError("Your Google Drive isn't mounted. Please run the above cell.")

assignment_dir = "/content/gdrive/MyDrive/workspace/BrainTumorDetection"  #@param{type:"string"}
assignment_dest = "/content/project"

# create symbolic link
!rm -f {assignment_dest}
!ln -s "{assignment_dir}" "{assignment_dest}"
print(f'Succesfully mapped (ln -s) "{assignment_dest}" -> "{assignment_dir}"')

# cd to linked dir
%cd -q {assignment_dest}
print(f'Succesfully changed directory (cd) to "{assignment_dest}"')

ModuleNotFoundError: No module named 'google.colab'

### Init

In [2]:
#@title ##Init

#@markdown Import required modules and observe your resources

# auto reload
%load_ext autoreload
%autoreload 2

# import
import torch
import torchvision
from PIL import Image
import matplotlib.pyplot as plt
import random
import os
import wandb
wandb.login()

from experiment import Experiment
from post_process import get_masks_from_heatmaps, present_masks, calc_iou
from data import DataGenerator, SegmentationGenerator

# Show resources
!nvidia-smi

wandb: Currently logged in as: sgvdan (use `wandb login --relogin` to force relogin)


Wed Jul 21 17:57:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     Off  | 00000000:14:00.0 Off |                  Off |
| 33%   28C    P8     4W / 260W |      0MiB / 48601MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Experiment

In [ ]:
# params
config = {
"model_name": "vgg19_seg",
#"data_train_class": DataGenerator,         "data_train_path": os.path.join(".", "Data", "Training"),
#"data_test_class": DataGenerator,          "data_test_path":  os.path.join(".", "Data", "Testing"),
"data_train_class": SegmentationGenerator, "data_train_path": os.path.join(".", "SegmentationData", "train"), 
"data_test_class": SegmentationGenerator,  "data_test_path":  os.path.join(".", "SegmentationData", "test-compact"),
"mri_type": "T1c", 
"epochs": 0,
"train_batch_size": 100,
"lr": 1e-4,
"heatmap_batch_size": 256,
"occlusion_size": (30,30),
"heat_layers" : [['features', layer_num] for layer_num in [0, 2, 5, 7, 10, 12, 14, 16, 19, 21, 23, 25, 28, 30, 32, 34]],
"ref_heat_layer": ['classifier', -1],
"heatmap_threshold" : 0.80,
"smallest_contour_len" : 30,
"shuffle_data": True,
"device": "cuda",
"gt_threshold": 0,
"attributes_to_log": ["model_name", "occlusion_size", "heatmap_threshold"],
"input_size": (256,256)
}

with wandb.init(project="BrainTumorDetection", config=config):
    ### load best model
    experiment = Experiment(config)
    iou = experiment.run()

loading vgg19_seg model...
loading best model state from: ./Models/vgg19_seg.pth


0it [00:00, ?it/s]


loading best model state from: ./Models/vgg19_seg.pth
Model's accuracy: 0.9
Creating heatmap...


100%|██████████| 256/256 [03:04<00:00,  1.39it/s]


Creating heatmap...


 95%|█████████▍| 243/256 [02:50<00:08,  1.52it/s]

### Debug